In [1]:
using Distributions, CSV, DataFrames, ProgressMeter
include("../../utils_slv.jl")

MethodOfLines (generic function with 1 method)

### Parameters

In [2]:
T = 1.0; r=0.05; S0 = 100.0; v0 = 0.4; α = 0.4; β = 0.9; ρ = 0.3; K = 100.0;
slvm = SABRSLV(S0::Float64, v0::Float64, α::Float64, β::Float64, ρ::Float64, r::Float64);
Π = [1.0 ρ; ρ 1.0]; cholΠ=cholesky(Π).L

drift(x) = [slvm.ω(abs(x[1]),abs(x[2])), slvm.μ(abs(x[2]))]
diffusion(x) = Diagonal(
        [slvm.m(abs(x[2]))*slvm.Γ(abs(x[1])), slvm.σ(abs(x[2]))]
        )*cholΠ

R = r;
driver(t, x, y, z) = (
    -r*y
)
terminal(x) = max(x[1]-K,0)
bsde = BSDE(T, [S0, v0], drift, diffusion, driver, terminal);

### Hagan's formula

In [3]:
function implied_volatility(t::Float64, f::Float64, v::Float64)
    # hagan
    if f != K
        numerator = (
            v * (
                1 + (
                    (1-β)^2/24 * v^2/(f*K)^(1-β) 
                    + 1/4 * ρ*β*α*v/(f*K)^((1-β)/2)
                    + (2-3*ρ^2)/24 * α^2
                    ) * t
                )
            )
        denominator = (
            (f*K)^((1-β)/2) * (
                1 + (1-β)^2/24 * log(f/K)^2
                + (1-β)^4/1920 * log(f/K)^4
                )
            )
        z = α/v * (f*K)^((1-β)/2) * log(f/K)
        x = log((sqrt(1-2*ρ*z+z^2)+z-ρ)/(1-ρ))
        return numerator/denominator * z/x
    else
        numerator = v * (
            1 + (
                (1-β)^2/24 * v^2/f^(2-2*β) 
                + 1/4 * ρ*β*α*v/f^(1-β)
                + (2-3*ρ^2)/24 * α^2
                ) * t
        )
        denominator = f^(1-β)
        return numerator/denominator
    end
end;

function BSvanilla(volatility, strike, expiry, spot, interest_rate, dividend_rate)
    forward = spot#*exp((interest_rate-dividend_rate)*expiry)
    sqrt_var = volatility * sqrt(expiry)
    if sqrt_var > 0.0
        d1 = log(forward/strike)/sqrt_var + sqrt_var/2
        d2 = d1 - sqrt_var
        call = forward*cdf(Normal(), d1) - strike*cdf(Normal(), d2)
        # put = -forward*cdf(Normal(), -d1) + exp(-(interest_rate-dividend_rate)*expiry)*strike*cdf(Normal(), -d2)
    else
        call = max(forward-strike, 0.0)
        # put = max(strike-forward, 0.0)
    end
    return exp(-(interest_rate-dividend_rate)*expiry)*call #put
end;

function price(t::Float64, f::Float64, v::Float64)
    return BSvanilla(implied_volatility(T-t, f, v), K, T-t, f, r, 0.0)
end;


## Experiment

In [4]:
Nₜs = [10, 20, 50, 100, 200]
header = vcat(["scheme", "measurement_type"], string.(Nₜs))
df = DataFrame([[],[],[],[],[],[],[]], header)
schemes = [
    [LawsonEuler(krylov=true, m=100), true],
    [NorsettEuler(krylov=true, m=100), true],
    [ETDRK2(krylov=true, m=100), true],
    [ETDRK3(krylov=true, m=100), true],
    [ETDRK4(krylov=true, m=100), true],
    [HochOst4(krylov=true, m=100), true]
    # [DP5(), false],
    # [RadauIIA5(), false]
]

6-element Vector{Vector{Any}}:
 [LawsonEuler{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]
 [NorsettEuler{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]
 [ETDRK2{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]
 [ETDRK3{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]
 [ETDRK4{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]
 [HochOst4{0, true, Val{:forward}, true, nothing}(true, 100, 0), true]

In [5]:
### designing grids
domain = [[0.0, 2*bsde.X0[1]], [0.0, 2*bsde.X0[2]]];

Nₗ = [150, 15]; Δₗ = (bsde.X0-[dom[1] for dom in domain])./Nₗ; 
Nᵣ = [150, 15]; Δᵣ = ([dom[2] for dom in domain]-bsde.X0)./Nᵣ;

In [6]:
spatial_grid = Array{AbstractGrid,1}(undef, 2)
g₁ = 5.0; g₂ = 5.0
spatial_grid[1] = TavellaRandallGrid(g₁, g₂, domain[1][1], bsde.X0[1], domain[1][2], Nₗ[1], Nᵣ[1])
spatial_grid[2] = Grid1D(
    vcat(domain[2][1]:Δₗ[2]:bsde.X0[2],(bsde.X0[2]+Δᵣ[2]):Δᵣ[2]:domain[2][2]),
    Nₗ[2]+Nᵣ[2]+1,
    Nₗ[2],
    Nᵣ[2]
)

Grid1D([0.0, 0.02666666666666667, 0.05333333333333334, 0.08, 0.10666666666666667, 0.13333333333333333, 0.16, 0.18666666666666668, 0.21333333333333335, 0.24  …  0.56, 0.5866666666666667, 0.6133333333333333, 0.64, 0.6666666666666666, 0.6933333333333334, 0.72, 0.7466666666666667, 0.7733333333333333, 0.8], 31, 15, 15)

In [7]:
d=2
@showprogress for attr in schemes
    sup_errs = zeros(Float64, length(Nₜs)); abs_errs = zeros(Float64, length(Nₜs)); runtimes = zeros(Float64, length(Nₜs))
    scheme = attr[1]
    EXPINT = attr[2]
    @showprogress for (ind, Nₜ) in enumerate(Nₜs)
        print(string("Nₜ=", Nₜ,":\n")); flush(stdout)
        exc_start = time()
        res = MethodOfLines(bsde, spatial_grid, Nₜ, scheme, EXPINT)
        exc_stop = time()
        sol = res[1]; s_grid = res[2]; 
        
        hagan_sol = zeros(prod([grid.N for grid in spatial_grid]), Nₜ+1)
        for (index, t) in enumerate((bsde.T/Nₜ).*(0:Nₜ))
            hagan_sol[:, index] = price.(Ref(T-t), s_grid[:,1], s_grid[:,2]);
        end
        abs_err = abs.(hagan_sol-sol)

        indmax = zeros(Int64,d)
        indmin = zeros(Int64,d)
        for dim in 1:d
            arr = findall(attr->(attr<1.2*bsde.X0[dim])&&(attr>0.8*bsde.X0[dim]), spatial_grid[dim].grid)
            indmin[dim] = minimum(arr)
            indmax[dim] = maximum(arr)
        end

        arr=zeros(indmax[1]-indmin[1]+1)
        supinds = zeros(Int,indmax[1]-indmin[1]+1,2)

        for (ind, pind) in enumerate(indmin[1]:indmax[1])
            arr[ind] = maximum(abs_err[(pind-1)*spatial_grid[2].N+indmin[2]+1:(pind-1)*spatial_grid[2].N+indmax[2],2:end])
            tmp=argmax(abs_err[(pind-1)*spatial_grid[2].N+indmin[2]+1:(pind-1)*spatial_grid[2].N+indmax[2],2:end])
            supinds[ind,1] = Int(tmp[1]+(pind-1)*spatial_grid[2].N+indmin[2]+1)
            supinds[ind,2] = Int(tmp[2])
        end
        
        sup_errs[ind] = maximum(arr)
        abs_errs[ind] = abs_err[spatial_grid[1].Nₗ*spatial_grid[2].N + spatial_grid[2].Nₗ+1,end]        
        runtimes[ind] = exc_stop - exc_start
        print(string("Sup Err=", sup_errs[ind],":\n")); flush(stdout)
        print(string("Y0(100,0.4)=", abs_errs[ind],":\n")); flush(stdout)
        print(string("Runtime[s]=", runtimes[ind],":\n")); flush(stdout)
    end
    schemename = split(split(string(scheme), '{')[1], '(')[1]
    row_sup = vcat([schemename, "Sup Err"], string.(sup_errs)); push!(df,row_sup);
    row_abs = vcat([schemename, "Abs Err"], string.(abs_errs)); push!(df,row_abs);
    row_run = vcat([schemename, "Runtime[s]"], string.(runtimes)); push!(df,row_run);
end
df |> CSV.write(string("SABR_exp_1030g5.csv"))

Nₜ=10:
Sup Err=1.9268712082537789:
Y0(100,0.4)=1.703275418639139:
Runtime[s]=36.37139391899109:
Nₜ=20:
Sup Err=0.3944606592999289:
Y0(100,0.4)=0.09161543342713685:
Runtime[s]=54.909815073013306:


Progress:  40%|████████████████▍                        |  ETA: 0:02:18

Nₜ=50:
Sup Err=0.05398743032552211:
Y0(100,0.4)=0.006197852217960431:
Runtime[s]=127.80985879898071:


Progress:  60%|████████████████████████▋                |  ETA: 0:02:27

Nₜ=100:
Sup Err=0.032650658196704097:
Y0(100,0.4)=0.0015506020070059634:
Runtime[s]=250.36766386032104:


...

Nₜ=100:
Sup Err=0.032588005415121746:
Y0(100,0.4)=0.0014079568158624767:
Runtime[s]=1185.4605140686035:


Progress:  80%|████████████████████████████████▊        |  ETA: 0:09:02

Nₜ=200:
Sup Err=0.032437875649971915:
Y0(100,0.4)=0.001239546048250162:
Runtime[s]=2410.866184949875:


Progress: 100%|█████████████████████████████████████████| Time: 1:16:19
Progress: 100%|█████████████████████████████████████████| Time: 4:24:25


"SABR_exp_1030g5.csv"

In [8]:
df

Row,scheme,measurement_type,10,20,50,100,200
,Any,Any,Any,Any,Any,Any,Any
1,LawsonEuler,Sup Err,1.9268712082537789,0.3944606592999289,0.05398743032552211,0.032650658196704097,0.03246096624194461
2,LawsonEuler,Abs Err,1.703275418639139,0.09161543342713685,0.006197852217960431,0.0015506020070059634,0.0013006068116521163
3,LawsonEuler,Runtime[s],36.37139391899109,54.909815073013306,127.80985879898071,250.36766386032104,503.9793179035187
4,NorsettEuler,Sup Err,1.934867763334596,0.38392105699835,0.051350691551789396,0.03159982791692295,0.03194409686036659
5,NorsettEuler,Abs Err,1.7047777926988523,0.08196235780944683,0.0033280616191913026,0.0002772881294941243,0.0006748299382710599
6,NorsettEuler,Runtime[s],29.91548991203308,52.66490387916565,130.9926438331604,245.1659209728241,494.20695996284485
7,ETDRK2,Sup Err,1.9391334039264603,0.3842611585074476,0.05139210419025897,0.03258766283636527,0.03243779542419345
8,ETDRK2,Abs Err,1.7085986964938389,0.08758575204510777,0.0055902695115914725,0.0014070340614438237,0.001239321465204668
9,ETDRK2,Runtime[s],55.86133098602295,106.29309391975403,245.76737594604492,514.8473510742188,995.9286239147186
